<a href="https://colab.research.google.com/github/ShivaShirsath/Plant-Disease-Diagnosis-Flask/blob/master/pddf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
! pip install flask-ngrok pyngrok gevent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 KB 22.6 MB/s eta 0:00:00


In [ ]:
! ngrok authtoken 2N6WNOvhe2XPK2SkHwaz7gwn0k0_21vYzWJsMBdgRaU7wYrkG

In [18]:
%cd /content
! git clone https://github.com/ShivaShirsath/Plant-Disease-Diagnosis-Flask.git
%cd /content/Plant-Disease-Diagnosis-Flask

/content
Cloning into 'Crop-Disease-Detection'...
remote: Enumerating objects: 308, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 308 (delta 16), reused 0 (delta 0), pack-reused 279
Receiving objects: 100% (308/308), 543.32 KiB | 5.38 MiB/s, done.
Resolving deltas: 100% (152/152), done.
Crop-Disease-Detection	       Plant-Leaf-Disease-Detection
Plant-Disease-Diagnosis-Flask  sample_data
/content/Crop-Disease-Detection
app.py			       Model.hdf5  README.md	     static	uploads
Crop-Diseases-Detection.ipynb  Procfile    requirements.txt  templates


In [25]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
from skimage import io
from tensorflow.keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer

# Define a flask app
app = Flask(__name__)

from flask_ngrok import run_with_ngrok

run_with_ngrok(app)

model =tf.keras.models.load_model('PlantDNet.h5',compile=False)

def model_predict(img_path, model):
    img = image.load_img(img_path, grayscale=False, target_size=(64, 64))
    show_img = image.load_img(img_path, grayscale=False, target_size=(64, 64))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = np.array(x, 'float32')
    x /= 255
    preds = model.predict(x)
    return preds


@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = os.getcwd()
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)

        # Make prediction
        preds = model_predict(file_path, model)
        print(preds[0])

        # x = x.reshape([64, 64]);
        disease_class = ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
                         'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight',
                         'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot',
                         'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot',
                         'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
        a = preds[0]
        ind=np.argmax(a)
        print('Prediction:', disease_class[ind])
        result=disease_class[ind]
        return result
    return None


if __name__ == '__main__':
    # app.run(port=5002, debug=True)

    # Serve the app with gevent
    #http_server = WSGIServer(('', 5000), app)
    #http_server.serve_forever()
    app.run()

Model loaded. Check http://127.0.0.1:5000/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://371f-35-192-173-82.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [17/Mar/2023 15:03:09] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2023 15:03:10] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2023 15:03:10] "GET /static/js/jquery.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2023 15:03:10] "GET /static/js/newjs.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2023 15:03:10] "GET /static/css/test.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2023 15:03:10] "GET /static/js/bootstrap.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2023 15:03:14] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2023 15:03:15] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 4s 4s/step


...

1/1 [==============================] - 0s 72ms/step


INFO:werkzeug:127.0.0.1 - - [17/Mar/2023 15:05:04] "POST /predict HTTP/1.1" 200 -


[3.26156104e-03 8.39496702e-02 3.63103151e-01 7.49359606e-04
 4.72869486e-01 5.16402943e-05 6.18042483e-04 1.20733705e-04
 3.67678695e-05 2.97283698e-02 4.19021817e-05 3.13076712e-02
 1.28487691e-05 3.57992295e-03 1.05690127e-02]
Prediction: Potato___healthy
1/1 [==============================] - 0s 62ms/step


INFO:werkzeug:127.0.0.1 - - [17/Mar/2023 15:05:21] "POST /predict HTTP/1.1" 200 -


[8.9186251e-06 4.9327785e-04 2.9611762e-04 6.6981638e-06 4.1969199e-04
 8.4187917e-08 2.1126493e-06 8.1736600e-04 5.6344114e-05 9.8674337e-04
 2.4694603e-07 2.6567121e-05 3.3731251e-07 2.1006515e-04 9.9667537e-01]
Prediction: Tomato_healthy
